# HyperDrive

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-nd-proj")
exp = Experiment(workspace=ws, name="udacity-proj")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: udacity-nd-proj
Azure region: eastus
Subscription id: 04620c09-8b1c-444e-a2bb-bdc3a7107fcb
Resource group: udacity-nd


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException

cluster_name = "pipeline-clust"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           vm_priority = 'lowpriority',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
params = {
    "C": uniform(0.0, 1.0),
    "max_iter": choice(50, 100, 150, 200)
}
ps = RandomParameterSampling(params)

# Specify a Policy
policy = BanditPolicy(
    slack_factor=0.1,
    evaluation_interval=1,
    delay_evaluation=5
)

if "training" not in os.listdir():
    print("Creating dir")
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory="./training",
    compute_target=compute_target,
    entry_script="train.py"
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_concurrent_runs=4,
    max_total_runs=20*len(params)
)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n C:', parameter_values[1])
print('\n max_iter:', parameter_values[3])

# save model
best_run.download_file("/outputs/model.joblib", "linreg_best_model.joblib")

Best Run Id:  HD_aa136b18-d92a-40e3-84a6-030505e1323b_7

 Accuracy: 0.915877640203933

 C: 0.6576290990355625

 max_iter: 150


# AutoML

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
data_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(data_path)

In [9]:
from sklearn.model_selection import train_test_split
import pandas as pd

from training.train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x_train, x_test, y_train, y_test = train_test_split(x, y)

train = pd.concat([x_train, y_train], axis=1)
train.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
8852,36,1,0,0,0,9,4,543,1,999,...,0,0,0,0,1,0,0,0,0,1
26446,30,1,0,1,0,7,5,53,6,999,...,0,0,0,0,1,0,0,0,0,0
18107,32,1,0,1,0,6,1,108,1,999,...,0,0,0,0,0,0,1,0,0,0
13418,46,0,0,0,0,6,3,53,2,999,...,1,0,0,0,0,0,1,0,0,0
14061,52,1,0,1,0,5,3,27,10,999,...,1,0,0,0,0,0,0,1,0,0


In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train,
    label_column_name="y",
    n_cross_validations=5)

In [11]:
# Submit your automl run
automl_run = exp.submit(automl_config, show_output=True)
RunDetails(hyperdrive_run).show()

Running on local machine
Parent Run ID: AutoML_7cb5943f-2eac-4753-a340-8a611497f553

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [12]:
# Retrieve and save your best automl model.
automl_run, fitted_automl_model = automl_run.get_output()
print(fitted_automl_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  l1_ratio=0.836734693877551,
                                                                                                  learning_rate='constant',
                                                                                                  loss='modified_huber',
                     

In [13]:
y_pred = fitted_automl_model.predict(x_test)

In [14]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, y_pred)
print("Test set accuracy:", acc)

Test set accuracy: 0.9176984705025492


In [15]:
import joblib

# save automl model
joblib.dump(fitted_automl_model, "fitted_automl_model.joblib")

['fitted_automl_model.joblib']

In [16]:
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

